# Analysis of tensorboard results
- view motion_statistic for automatic analysis.
- due to history code problem, cannot show dataset name in table directly (name for ss001 and ss002, dataset_name for others), modify summary() to support it.

In [19]:
import os
import sys
sys.path.insert(0,os.path.expanduser('~/git/torchseg'))
print(sys.path)

%reload_ext autoreload
%autoreload 2
from torchseg.utils.summary_to_jupyter import summary,dump

['/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/git/torchseg/notebooks', '/home/yzbx/miniconda3/envs/new/lib/python38.zip', '/home/yzbx/miniconda3/envs/new/lib/python3.8', '/home/yzbx/miniconda3/envs/new/lib/python3.8/lib-dynload', '', '/home/yzbx/.local/lib/python3.8/site-packages', '/home/yzbx/miniconda3/envs/new/lib/python3.8/site-packages', '/home/yzbx/miniconda3/envs/new/lib/python3.8/site-packages/IPython/extensions', '/home/yzbx/.ipython']


In [21]:
tags=['train/iou','val/iou']
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
descripe=['note','n_epoch','input_shape']
# attention_type aux_backbone filter_feature
invalid_param_list=['dir','log_time','root_path',
                    'test_path', 'train_path', 'val_path',
                    'use_optical_flow','flow_backbone',
                    'ignore_outOfRoi','exception_value',
                    'cityscapes_split','txt_path',
                    'decode_main_layer','gpu','ngpus_per_node',
                    'rank']
note_gtags=[['dataset','note'],['dataset','note']]
note_gtags=None
notes=['mox001','mox002']
# notes=['ss042_attention','ss042_resnet','ss042_vgg']
# notes=['ss059']
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,descripe=descripe,
     invalid_param_list=invalid_param_list,delete_nan=False,dump_group=False,
     note_gtags=note_gtags)

### mox001 batch_size,dataset_name,freeze_layer,midnet_scale,n_epoch,net_name,save_model,txt_note,upsample_layer,use_sync_bn,actual_step
|    |   n_epoch |   freeze_layer | txt_note        | save_model   |   actual_step |   batch_size |   train/iou |   midnet_scale |   upsample_layer | note   | dataset_name        | input_shape   |   val/iou | use_sync_bn   | net_name      |
|---:|----------:|---------------:|:----------------|:-------------|--------------:|-------------:|------------:|---------------:|-----------------:|:-------|:--------------------|:--------------|----------:|:--------------|:--------------|
|  0 |         5 |              1 | cityscapes_fine | True         |             5 |            4 |    0.779446 |              1 |                1 | mox001 | cityscapes_category | [480, 480]    |  0.765276 | True          | PSPUNet       |
|  1 |        50 |              1 | huawei          | False        |            50 |            4 |    0.80292  |              1 |          

In [4]:
rootpath=os.path.expanduser('~/tmp/logs/pytorch')
tags=['train/iou','val/iou']
descripe=['note','n_epoch']
notes=['today','recent']
note_gtags=[['dataset_name','log_time','net_name'],
           ['dataset_name','log_time','net_name']]
recent_log_number=100
tasks=dump(tags=tags,rootpath=rootpath,notes=notes,note_gtags=note_gtags,sort_tags=['dataset_name','val/iou'],
     descripe=descripe,delete_nan=False,dump_group=False,recent_log_number=recent_log_number)

|    |   n_epoch | log_time              | note   |   train/iou | dataset_name   |   val/iou | net_name      |
|---:|----------:|:----------------------|:-------|------------:|:---------------|----------:|:--------------|
|  2 |        50 | 2020-12-23___13-42-06 | mox001 |    0.746095 | huawei         |  0.558589 | DeepLabV3Plus |
|  1 |        50 | 2020-12-23___13-45-14 | mox001 |    0.790604 | huawei         |  0.611118 | DeepLabV3Plus |
|  0 |        50 | 2020-12-23___12-31-57 | mox001 |    0.80292  | huawei         |  0.624322 | PSPUNet       |


|    |   val/iou | dir                                                                                      |
|---:|----------:|:-----------------------------------------------------------------------------------------|
|  2 |  0.558589 | /home/yzbx/tmp/logs/pytorch/DeepLabV3Plus/huawei/mox001/2020-12-23___13-42-06/config.txt |
|  1 |  0.611118 | /home/yzbx/tmp/logs/pytorch/DeepLabV3Plus/huawei/mox001/2020-12-23___13-45-14/config.txt |
|  

In [36]:
print(tasks.columns)
print(tasks['decode_main_layer'])
import math
import numpy as np
a=tasks['decode_main_layer']
if np.nan in tasks['decode_main_layer']:
    print('a is nan')
else:
    print('a is not nan')

import pandas as pd
print(pd.isna(tasks['accumulate']).any())
print(pd.isnull(tasks['accumulate']).any())

Index(['accumulate', 'attention_type', 'augmentation', 'augmentations_blur',
       'aux_base_weight', 'auxnet_layer', 'auxnet_type', 'backbone_freeze',
       'backbone_name', 'backbone_pretrained',
       ...
       'use_lr_mult', 'use_none_layer', 'use_reg', 'use_rotate',
       'vertical_flip', 'with_edge', 'train/iou', 'val/iou', 'dir',
       'log_time'],
      dtype='object', length=104)
0    1.0
1    1.0
2    NaN
Name: decode_main_layer, dtype: float64
a is not nan
False
False
